In [129]:
import sys
import cairocffi as cairo
import re
import Bio
import matplotlib as mp
from Bio import SeqIO

In [185]:
mot=""
with open('Fig_1_motifs.txt', "r") as m:
    m_ref = []
    m_raw = []
    for line in m:
        x = line.split('\t')[0]
        y = x.strip()
        m_raw.append(y)
        for letter in y:
            mot = mot + reg_ref[letter].lower()
        m_ref.append("(?=("+mot+"))")
        mot = ""
        
print(m_ref)

['(?=([uct]gc[uct]))', '(?=(gcatg))', '(?=(catag))', '(?=([uct][uct][uct][uct][uct][uct][uct][uct][uct][uct]))']


In [115]:
m_ref = ['ygcy', 'GCAUG', 'catag', 'YYYYYYYYYY']

reg_ref = {"y": "[uct]", "g": "g", "c": "c", "t":"t", "a": "a", "u": "t", "n": "[uctag]", "Y": "[UCT]", "G": "G","T":"T", "C": "C", "A": "A", "U": "T", "N": "[UCTAG]"}

m_ref2 = []
mot = ""

for cha in m_ref:
    for letter in cha:
        mot = mot + reg_ref[letter].lower()
    m_ref2.append("(?=("+mot+"))")
    mot = ""

In [ ]:
#m_ref2[1].lower()
#sequence.lower()
#tertest=re.finditer(m_ref2[0].lower(), sequence.lower())


In [116]:
m_ref2

['(?=([uct]gc[uct]))',
 '(?=(gcatg))',
 '(?=(catag))',
 '(?=([uct][uct][uct][uct][uct][uct][uct][uct][uct][uct]))']

In [197]:
max_len = 0
num_record=0
for record in SeqIO.parse('Figure_1.fasta', "fasta"):
    num_record+=1
    if len(record) > max_len:
        max_len = len(record)

image = cairo.SVGSurface("Figure_1.svg", (max_len+100), (num_record*200))
#scale to len of read
context = cairo.Context(image)
motif_color=0
count=0
point_x = 10
point_y = 75

# Cyan, Blue, Violet, Purple-Red, Light Gray, Dark Gray, Red, Orange, Yellow, Green,       
RGB = [[0,0.8,0.8],[0,0,0.8],[0.6,0,0.6],[0.8,0,0],[0.25,0.25,0.25],
       [0.5,0.5,0.5],[0.6,0,0.3],[0.8,0.4,0],[0.8,0,0.8],[0,0.8,0]]

for record in SeqIO.parse('Figure_1.fasta', "fasta"):   
    header = str('>'+record.id)
    sequence = str(record.seq)
    back =str(record.seq)
    exon = re.findall(r'[A-Z]+', sequence)
    #setting aesthetics
    context.set_line_width(2)
    context.set_source_rgb(0, 0, 0)
    #header
    context.move_to(point_x, point_y)
    context.set_font_size(10)
    context.show_text(header)
    #drawing a line for the sequence
    context.move_to(point_x+50, point_y)
    context.line_to((len(sequence)+point_x+50), point_y)
    context.move_to(point_x, point_y-30)
    #drawing a rectangle for the exon
    exon_start = sequence.find(exon[0]) + 1 + point_x
    context.rectangle(exon_start, point_y-15, len(exon[0]), 30)
    context.stroke()
    
    for m in m_ref2:
        sequence = str(record.seq)
        pos = 1
        loc = re.findall(m, sequence.lower())
        firstPass = True
        motif_color=m_ref2.index(m)
        context.set_source_rgb(RGB[motif_color][0], RGB[motif_color][1], RGB[motif_color][2])
        context.stroke()
        if len(loc)==0:
            context.set_font_size(10)
            context.move_to(point_x, point_y + (len(m_ref)*9) + (motif_color * 10))
            string= str("There are 0 occurances of "+ m_raw[motif_color]) + " in this record"
            context.show_text(string)
            
        for x in range(len(loc)):

            motifs = sequence.lower().find(loc[x]) + pos
    
            context.set_line_width(1+ (len(m)/10))

            context.move_to(motifs+point_x+50 + (len(loc)/2), point_y+10)
            context.line_to(motifs+point_x+50 + (len(loc)/2), point_y-10)

            context.stroke()
        
            pos += sequence.lower().find(loc[x]) +1
            #i think the problem i'm havig is that it's finding one and then not marking the others
            
            sequence = sequence[(sequence.lower().find(loc[x])+1):]    
            if firstPass == True: 
                context.set_font_size(10)
                context.move_to(point_x, point_y + (len(m_ref)*9) + (motif_color * 10))
                string= str("There are " + str(len(loc)) + " occurances of "+ m_raw[motif_color]) + " in this record"
                context.show_text(string)
                firstPass = False

    point_y += (len(m_ref)*25)
    count+=1

image.finish()

In [147]:
list.index()

10

In [83]:
sequence[sequence.lower().find('tgcc')+1:]

'gccttttgggtaactctttagtattttagcttctagttcctcctctctgccctgttctgctggtactttcttccaagaaaaaggaactcttaagaggatgagagttggctaaaacaaaaaataatctttctttcaactgtggcactaccctgggttgcccgtctgtgttttattttccaaaggcaaaaaaaccattatgctatgttaagctaatttcaatacaaggcacacttacttgcttcagactctgctttgctgttgaacttgtttcccttttaattttgttttaaattgtacttttgcttttttagtagctaataacattcggctttttaattttttgaatgttactaactgttccctgttatcccttcccttgttttttaatttgcttaaagATGGTACTACCACTAAAGCGGAGGgtaagcggtttgtttctgtgattgaggtgtgtctcactttctctggtcctctgttctcaccgagtgttgcccagtacccacttgtcctcaccaaggtccctgcatgagatgttgcacgctgtttctcctcctctcttttcattctgtcgccattgggtttttgccctacctctacctgagttctccttccctttcttgttcagtctttgttcttgtgaattttagcactaaagtatgctgcgtgtggtttttattgctgagtccacactctggggaggagcattgatgtcacactgtgtgctactgtcctcatggactgcctgtcttgggttttattactcctgaa'

In [118]:
count

120